# データ
次の辞書ファイルを作成する

## 姓名辞書
- oss.json
オープンソースの辞書から作成した，漢字・アルファベットとカタカナのペアデータ
- seimei.json
ダウンロードしたWebデータ（Wikipedia）から作成した漢字・アルファベットとカタカナのペアのデータ

## 単漢字辞書
- tankanji.json
オープンソースの辞書から作成した，単漢字とカタカナのペアデータ

## 異体字辞書
- itaiji.json
オープンソースの辞書から作成した，異体字の漢字のペアデータ




# データを準備する

辞書用及び。AI用のデータを，オープンソースのデータからダウンロードし，整形する。
使用するデータソースは次のとおり      

## データソース一覧

### オープンソース辞書データ
- [canna](https://osdn.dl.osdn.net/canna/)
- [freewnn](https://osdn.dl.osdn.net/freewnn/)
- [ipadic](https://taku910.github.io/mecab/)
- [kakasi](http://kakasi.namazu.org/)
- [mj](https://moji.or.jp/)
- [mozc](https://github.com/google/mozc/)
- [neologd](https://github.com/neologd/mecab-ipadic-neologd/)
- [skk](https://github.com/skk-dev/dict/)

### Webデータ
- [wikipedia](https://dumps.wikimedia.org/)

## データ形式について
データは，単語辞書，単漢字辞書，異体字辞書をそれぞれのデータソースから作成する。
作成するデータフォーマットはJSON形式であり，項目については次のとおり。

### 姓名辞書
単語漢字をキーとし，その読みのカタカナをキーとするハッシュテーブルをバリューとして追加する。またカタカナにはそれぞれdicsというキーで，漢字とカタカナのペアが存在する，データソースの一覧を保持している。
```json
{ 
    "単語漢字１" :{
        "カタカナ１":{
            "dics":[
                "freewnn",
                "canna"
            ]
        },
        "カタカナ２":{
            "dics":{
                "canna"
            }
        }
    },
    "単語漢字２":{
        ..
    }
..

```

### 単漢字辞書
単漢字をキーとし，その読みのカタカナをキーとするハッシュテーブルをバリューとして追加する。またカタカナにはそれぞれdicsというキーで，漢字とカタカナのペアが存在する，データソースの一覧を保持している。
```json
{ 
    "単漢字１" :{
        "カタカナ１":{
            "dics":[
                "freewnn",
                "canna"
            ]
        },
        "カタカナ２":{
            "dics":{
                "canna"
            }
        }
    },
    "単漢字２":{
        ..
    }
..

```


### 異体字辞書
単漢字をキーとし，その異体字である単漢字をキーとするハッシュテーブルをバリューとして追加する。また異体字である単漢字にはそれぞれdicsというキーで，漢字とカタカナのペアが存在する，データソースの一覧を保持している。

```json
{ 
    "単漢字１" :{
        "異体字１":{
            "dics":[
                "kakasi",
                "mozc"
            ]
        },
        "異体字２":{
            "dics":{
                "skk"
            }
        }
    },
    "単漢字２":{
        ..
    }
..

```

## 事前準備
pythonで必要なライブラリ群をインストールしておく

In [1]:
# warning off
import warnings
warnings.filterwarnings('ignore')

In [2]:
#!pip install -r ../requirements.txt;

%cd oss

/home/analysis01/src/kanjikana-model/dict/oss


In [3]:
!pip install wget


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


# データをダウンロードし，漢字とカタカナを抜き出す
あらかじめ，nkfコマンドをインストールしておく必要がある。

## canna
cannnaからは，単語辞書と単漢字辞書を作成する。単漢字辞書は単語辞書から一文字の漢字を抜き出したもの。


In [4]:

%cd canna
#!wget -nc -q "https://osdn.dl.osdn.net/canna/9565/Canna37p3.tar.bz2" --no-check-certificate;
#!bunzip2 Canna37p3.tar.bz2;
#!tar xfp Canna37p3.tar;
#!bzip2 Canna37p3.tar;
!rm -rf v3.7p3.tar.gz
!wget "https://github.com/netsphere-labs/canna/archive/refs/tags/v3.7p3.tar.gz"


/home/analysis01/src/kanjikana-model/dict/oss/canna
--2025-05-30 13:38:30--  https://github.com/netsphere-labs/canna/archive/refs/tags/v3.7p3.tar.gz
github.com (github.com) をDNSに問いあわせています... 20.27.177.113
github.com (github.com)|20.27.177.113|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 302 Found
場所: https://codeload.github.com/netsphere-labs/canna/tar.gz/refs/tags/v3.7p3 [続く]
--2025-05-30 13:38:30--  https://codeload.github.com/netsphere-labs/canna/tar.gz/refs/tags/v3.7p3
codeload.github.com (codeload.github.com) をDNSに問いあわせています... 20.27.177.114
codeload.github.com (codeload.github.com)|20.27.177.114|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 特定できません [application/x-gzip]
`v3.7p3.tar.gz' に保存中

v3.7p3.tar.gz           [ <=>                ]   1.43M  8.49MB/s 時間 0.2s       

2025-05-30 13:38:30 (8.49 MB/s) - `v3.7p3.tar.gz' へ保存終了 [1497279]



In [5]:
!tar zxvfp v3.7p3.tar.gz

canna-3.7p3/
canna-3.7p3/.cvsignore
canna-3.7p3/CHANGES.jp
canna-3.7p3/COPYRIGHT
canna-3.7p3/Canna.conf
canna-3.7p3/Canna.conf.dist
canna-3.7p3/ChangeLog
canna-3.7p3/INSTALL
canna-3.7p3/INSTALL.jp
canna-3.7p3/Imakefile
canna-3.7p3/OCHANGES.jp
canna-3.7p3/README
canna-3.7p3/README.jp
canna-3.7p3/RKCCONF.jp
canna-3.7p3/WHATIS
canna-3.7p3/WHATIS.jp
canna-3.7p3/accanna.h.in
canna-3.7p3/canna/
canna-3.7p3/canna/.cvsignore
canna-3.7p3/canna/Imakefile
canna-3.7p3/canna/ccompat.h
canna-3.7p3/canna/jrkanji.h
canna-3.7p3/canna/keydef.h
canna-3.7p3/canna/mfdef.h
canna-3.7p3/canna/net.h
canna-3.7p3/canna/patchlevel.h
canna-3.7p3/canna/protodefs.h
canna-3.7p3/canna/rkcapi.h
canna-3.7p3/canna/symbolname.h
canna-3.7p3/canna/sysdep.h
canna-3.7p3/canna/widedef.h
canna-3.7p3/canuum/
canna-3.7p3/canuum/.cvsignore
canna-3.7p3/canuum/COPYRIGHT
canna-3.7p3/canuum/Imakefile
canna-3.7p3/canuum/README.copyright
canna-3.7p3/canuum/README.jp
canna-3.7p3/canuum/acconfig.h
canna-3.7p3/canuum/buffer.h
canna-3.7p3/c

In [6]:
# 単語辞書を作成する，漢字とカタカナのペアのCSV形式
!python extract_canna.py --indir canna-3.7p3/dic/ideo/pubdic --outfile extract.txt
!python words.py --indir "canna-3.7p3/dic/ideo/words" --outfile words.txt
!cat extract.txt words.txt > dict.txt


{
  "indir": "canna-3.7p3/dic/ideo/pubdic",
  "outfile": "extract.txt"
}
{
  "indir": "canna-3.7p3/dic/ideo/words",
  "outfile": "words.txt"
}


In [7]:
# 単漢字辞書を作成する。単語辞書から一文字の漢字を抜き出す
!python ../tankan.py --infile dict.txt --outfile tankanji.txt


{
  "infile": "dict.txt",
  "outfile": "tankanji.txt"
}


In [8]:
# cannaのダウンロードデータを削除する
!rm -rf canna-3.7p3
%cd ..


/home/analysis01/src/kanjikana-model/dict/oss


## freewnn
freewnnからは，単語辞書と単漢字辞書を作成する。単漢字辞書は単語辞書から一文字の漢字を抜き出したもの。


In [9]:
%cd freewnn
!rm -rf master*
#!wget -nc -q https://osdn.dl.osdn.net/freewnn/63271/FreeWnn-1.1.1-a023.tar.gz --no-check-certificate
#!tar zxfp FreeWnn-1.1.1-a023.tar.gz


!wget https://github.com/MItsutoshiNAKANO/FreeWnn/archive/refs/heads/master.zip


/home/analysis01/src/kanjikana-model/dict/oss/freewnn
--2025-05-30 13:38:33--  https://github.com/MItsutoshiNAKANO/FreeWnn/archive/refs/heads/master.zip
github.com (github.com) をDNSに問いあわせています... 20.27.177.113
github.com (github.com)|20.27.177.113|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 302 Found
場所: https://codeload.github.com/MItsutoshiNAKANO/FreeWnn/zip/refs/heads/master [続く]
--2025-05-30 13:38:33--  https://codeload.github.com/MItsutoshiNAKANO/FreeWnn/zip/refs/heads/master
codeload.github.com (codeload.github.com) をDNSに問いあわせています... 20.27.177.114
codeload.github.com (codeload.github.com)|20.27.177.114|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 特定できません [application/zip]
`master.zip' に保存中

master.zip              [   <=>              ]   4.40M  9.08MB/s 時間 0.5s       

2025-05-30 13:38:34 (9.08 MB/s) - `master.zip' へ保存終了 [4615276]



In [10]:
!unzip master.zip

Archive:  master.zip
08f38ed1aaaf8ca4787b897863ac8f3bac07ec78
   creating: FreeWnn-master/
  inflating: FreeWnn-master/CONTRIBUTORS  
  inflating: FreeWnn-master/COPYING.LIB  
  inflating: FreeWnn-master/COPYRIGHT  
  inflating: FreeWnn-master/COPYRIGHT-j  
  inflating: FreeWnn-master/ChangeLog  
  inflating: FreeWnn-master/ChangeLog.en  
  inflating: FreeWnn-master/ChangeLog.old  
   creating: FreeWnn-master/Contrib/
   creating: FreeWnn-master/Contrib/dic/
   creating: FreeWnn-master/Contrib/dic/gerodic/
  inflating: FreeWnn-master/Contrib/dic/gerodic/COPYRIGHT  
  inflating: FreeWnn-master/Contrib/dic/gerodic/GERODIC  
  inflating: FreeWnn-master/Contrib/dic/gerodic/INSTALL  
  inflating: FreeWnn-master/Contrib/dic/gerodic/Makefile.in  
  inflating: FreeWnn-master/Contrib/dic/gerodic/README  
  inflating: FreeWnn-master/Contrib/dic/gerodic/Version  
  inflating: FreeWnn-master/Contrib/dic/gerodic/g-jinmei.u  
  inflating: FreeWnn-master/INSTALL  
  inflating: FreeWnn-master/INSTALL.

In [11]:
# 単語辞書を作成する，漢字とカタカナのペアのCSV形式
!python extract_freewnn.py --infile FreeWnn-master/PubdicPlus/pubdic.p --outfile dict.txt
!rm -rf FreeWnn-master

{
  "infile": "FreeWnn-master/PubdicPlus/pubdic.p",
  "outfile": "dict.txt"
}


In [12]:
# 単漢字辞書を作成する。単語辞書から一文字の漢字を抜き出す
!python ../tankan.py --infile dict.txt --outfile tankanji.txt


{
  "infile": "dict.txt",
  "outfile": "tankanji.txt"
}


In [13]:

# freewnnのダウンロードデータを削除する
!rm -rf FreeWnn-master
%cd ..


/home/analysis01/src/kanjikana-model/dict/oss



## ipadic
ipadicからは，単語辞書と単漢字辞書を作成する。単漢字辞書は単語辞書から一文字の漢字を抜き出したもの。

In [14]:
%cd ipadic
!rm -rf *gz
!wget -nc -q --no-check-certificate  "https://jaist.dl.sourceforge.net/project/mecab/mecab-ipadic/2.7.0-20070801/mecab-ipadic-2.7.0-20070801.tar.gz" 
#!tar zxfp mecab-ipadic-2.7.0-20070801.tar.gz



/home/analysis01/src/kanjikana-model/dict/oss/ipadic


In [15]:
!tar zxfp mecab-ipadic-2.7.0-20070801.tar.gz


In [16]:
# 単語辞書を作成する，漢字とカタカナのペアのCSV形式
!python extract_ipadic.py --indir="mecab-ipadic-2.7.0-20070801/" --outfile="dict.txt"



{
  "indir": "mecab-ipadic-2.7.0-20070801/",
  "outfile": "dict.txt"
}
mecab-ipadic-2.7.0-20070801/Adj.csv
mecab-ipadic-2.7.0-20070801/Adnominal.csv
mecab-ipadic-2.7.0-20070801/Adverb.csv
mecab-ipadic-2.7.0-20070801/Auxil.csv
mecab-ipadic-2.7.0-20070801/Conjunction.csv
mecab-ipadic-2.7.0-20070801/Filler.csv
mecab-ipadic-2.7.0-20070801/Interjection.csv
mecab-ipadic-2.7.0-20070801/Noun.adjv.csv
mecab-ipadic-2.7.0-20070801/Noun.adverbal.csv
mecab-ipadic-2.7.0-20070801/Noun.csv
mecab-ipadic-2.7.0-20070801/Noun.demonst.csv
mecab-ipadic-2.7.0-20070801/Noun.nai.csv
mecab-ipadic-2.7.0-20070801/Noun.name.csv
mecab-ipadic-2.7.0-20070801/Noun.number.csv
mecab-ipadic-2.7.0-20070801/Noun.org.csv
mecab-ipadic-2.7.0-20070801/Noun.others.csv
mecab-ipadic-2.7.0-20070801/Noun.place.csv
mecab-ipadic-2.7.0-20070801/Noun.proper.csv
mecab-ipadic-2.7.0-20070801/Noun.verbal.csv
mecab-ipadic-2.7.0-20070801/Others.csv
mecab-ipadic-2.7.0-20070801/Postp-col.csv
mecab-ipadic-2.7.0-20070801/Postp.csv
mecab-ipadic-2

In [17]:
# 単漢字辞書を作成する。単語辞書から一文字の漢字を抜き出す
!python ../tankan.py --infile dict.txt --outfile tankanji.txt



{
  "infile": "dict.txt",
  "outfile": "tankanji.txt"
}


In [18]:
# ダウンロードデータを削除する
!rm -rf mecab-ipadic-2.7.0-20070801
%cd ..


/home/analysis01/src/kanjikana-model/dict/oss


## kakasi
kakasiからは，単語辞書と単漢字辞書，異体字辞書を作成する。単漢字辞書は単語辞書から一文字の漢字を抜き出したもの。

In [19]:
%cd kakasi
!rm -rf *gz
!wget -nc -q http://kakasi.namazu.org/stable/kakasi-2.3.6.tar.gz --no-check-certificate
!tar zxfp kakasi-2.3.6.tar.gz


/home/analysis01/src/kanjikana-model/dict/oss/kakasi


In [20]:
# 単語辞書を作成する，漢字とカタカナのペアのCSV形式
!python dict_kakasi.py --infile="kakasi-2.3.6/kakasidict" --outfile="dict.txt"



In [21]:
# 単漢字辞書を作成する。単語辞書から一文字の漢字を抜き出す
!python ../tankan.py --infile dict.txt --outfile tankanji.txt

{
  "infile": "dict.txt",
  "outfile": "tankanji.txt"
}


In [22]:
# 異体字辞書を作成する
!python  itaiji_kakasi.py --infile="kakasi-2.3.6/itaijidict" --outfile="itaiji.txt"

In [23]:
# ダウンロードデータを削除する
!rm -rf kakasi-2.3.6
%cd ..

/home/analysis01/src/kanjikana-model/dict/oss


## mj
mjからは単漢字辞書を作成する。データはExcel形式で提供されているので，Excel形式をCSV形式へ変換する[xlsx2csv](https://github.com/dilshod/xlsx2csv)を利用する。

In [24]:
%cd mj
!rm -rf *xlsx
!wget -nc -q https://moji.or.jp/wp-content/uploads/2024/01/mji.00602.xlsx


/home/analysis01/src/kanjikana-model/dict/oss/mj


In [25]:
!pip install xlsx2csv


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [26]:
# excel形式からcsvへ変換する
!xlsx2csv mji.00602.xlsx mj.csv

In [27]:
# 単漢字辞書を作成する。
!python extract_xlsx.py --infile mj.csv --outfile tankanji.txt

{
  "infile": "mj.csv",
  "outfile": "tankanji.txt"
}
cnt=105175
len=51673


In [28]:
# ダウンロードしたファイルと不要ファイルを削除する
!rm -rf *.csv
%cd ..

/home/analysis01/src/kanjikana-model/dict/oss


## mozc
mozcからは，単語辞書と単漢字辞書，異体字辞書を作成する。単漢字辞書は単語辞書から一文字の漢字を抜き出したもの。



In [29]:
%cd mozc
!rm -rf *zip
!rm -rf *gz
!rm -rf mozc-*
!wget -nc -q "https://github.com/google/mozc/archive/refs/tags/2.31.5712.102.zip" --no-check-certificate


/home/analysis01/src/kanjikana-model/dict/oss/mozc


In [30]:
!unzip  2.31.5712.102.zip

Archive:  2.31.5712.102.zip
58ebfadae04bc46c46e70edc9662347a29d8b7bc
   creating: mozc-2.31.5712.102/
  inflating: mozc-2.31.5712.102/.clang-format  
  inflating: mozc-2.31.5712.102/.gitattributes  
   creating: mozc-2.31.5712.102/.github/
   creating: mozc-2.31.5712.102/.github/ISSUE_TEMPLATE/
  inflating: mozc-2.31.5712.102/.github/ISSUE_TEMPLATE/bug_report.md  
  inflating: mozc-2.31.5712.102/.github/ISSUE_TEMPLATE/build_error.yml  
  inflating: mozc-2.31.5712.102/.github/ISSUE_TEMPLATE/discussions.yml  
  inflating: mozc-2.31.5712.102/.github/ISSUE_TEMPLATE/typing_issue.yml  
  inflating: mozc-2.31.5712.102/.github/pull_request_template.md  
   creating: mozc-2.31.5712.102/.github/workflows/
  inflating: mozc-2.31.5712.102/.github/workflows/android.yaml  
  inflating: mozc-2.31.5712.102/.github/workflows/lint.yaml  
  inflating: mozc-2.31.5712.102/.github/workflows/linux.yaml  
  inflating: mozc-2.31.5712.102/.github/workflows/macos.yaml  
  inflating: mozc-2.31.5712.102/.github/wo

In [31]:
# 単語辞書を作成する
!python extract_mozc.py --indir="mozc-2.31.5712.102/src/data/dictionary_oss/" --outfile="dict1.txt"
!python extract_oss.py --indir="mozc-2.31.5712.102/src/data/oss/" --outfile="dict2.txt"
!cat dict1.txt dict2.txt > dict.txt
!rm -rf dict1.txt dict2.txt


{
  "indir": "mozc-2.31.5712.102/src/data/dictionary_oss/",
  "outfile": "dict1.txt"
}
mozc-2.31.5712.102/src/data/dictionary_oss/dictionary00.txt
mozc-2.31.5712.102/src/data/dictionary_oss/dictionary01.txt
mozc-2.31.5712.102/src/data/dictionary_oss/dictionary02.txt
mozc-2.31.5712.102/src/data/dictionary_oss/dictionary03.txt
mozc-2.31.5712.102/src/data/dictionary_oss/dictionary04.txt
mozc-2.31.5712.102/src/data/dictionary_oss/dictionary05.txt
mozc-2.31.5712.102/src/data/dictionary_oss/dictionary06.txt
mozc-2.31.5712.102/src/data/dictionary_oss/dictionary07.txt
mozc-2.31.5712.102/src/data/dictionary_oss/dictionary08.txt
mozc-2.31.5712.102/src/data/dictionary_oss/dictionary09.txt
{
  "indir": "mozc-2.31.5712.102/src/data/oss/",
  "outfile": "dict2.txt"
}


In [32]:
# 単漢字辞書を作成する
!python extract_single.py --indir="mozc-2.31.5712.102/src/data/single_kanji/" --outfile="tankanji.txt"


{
  "indir": "mozc-2.31.5712.102/src/data/single_kanji/",
  "outfile": "tankanji.txt"
}
mozc-2.31.5712.102/src/data/single_kanji//single_kanji.tsv


In [33]:
# 異体字辞書を作成する 
!python extract_variant.py --indir="mozc-2.31.5712.102/src/data/single_kanji/" --outfile="itaiji.txt"

{
  "indir": "mozc-2.31.5712.102/src/data/single_kanji/",
  "outfile": "itaiji.txt"
}
mozc-2.31.5712.102/src/data/single_kanji//variant_rule.txt


In [34]:
# 不要ファイルを削除する
!rm -rf mozc-2.31.5712.102
%cd ..


/home/analysis01/src/kanjikana-model/dict/oss


## neologd
ipadicからは，単語辞書と単漢字辞書を作成する。単漢字辞書は単語辞書から一文字の漢字を抜き出したもの。


In [35]:
%cd neologd
!rm -rf *gz
!wget -nc -q https://github.com/neologd/mecab-ipadic-neologd/archive/refs/tags/v0.0.7.tar.gz --no-check-certificate
!tar zxf v0.0.7.tar.gz
!xz --decompress  mecab-ipadic-neologd-0.0.7/seed/*xz

/home/analysis01/src/kanjikana-model/dict/oss/neologd


In [36]:
# 単語辞書を作成する，漢字とカタカナのペアのCSV形式
!python extract_neologd.py --indir="mecab-ipadic-neologd-0.0.7/seed" --outfile="dict.txt"



{
  "indir": "mecab-ipadic-neologd-0.0.7/seed",
  "outfile": "dict.txt"
}
mecab-ipadic-neologd-0.0.7/seed/mecab-user-dict-seed.20200820.csv
mecab-ipadic-neologd-0.0.7/seed/neologd-adjective-exp-dict-seed.20151126.csv
mecab-ipadic-neologd-0.0.7/seed/neologd-adjective-std-dict-seed.20151126.csv
mecab-ipadic-neologd-0.0.7/seed/neologd-adjective-verb-dict-seed.20160324.csv
mecab-ipadic-neologd-0.0.7/seed/neologd-adverb-dict-seed.20150623.csv
mecab-ipadic-neologd-0.0.7/seed/neologd-common-noun-ortho-variant-dict-seed.20170228.csv
mecab-ipadic-neologd-0.0.7/seed/neologd-date-time-infreq-dict-seed.20190415.csv
mecab-ipadic-neologd-0.0.7/seed/neologd-ill-formed-words-dict-seed.20170127.csv
mecab-ipadic-neologd-0.0.7/seed/neologd-interjection-dict-seed.20170216.csv
mecab-ipadic-neologd-0.0.7/seed/neologd-noun-sahen-conn-ortho-variant-dict-seed.20160323.csv
mecab-ipadic-neologd-0.0.7/seed/neologd-proper-noun-ortho-variant-dict-seed.20161110.csv
mecab-ipadic-neologd-0.0.7/seed/neologd-quantity-in

In [37]:
# 単漢字辞書を作成する。単語辞書から一文字の漢字を抜き出す
!python ../tankan.py --infile dict.txt --outfile tankanji.txt




{
  "infile": "dict.txt",
  "outfile": "tankanji.txt"
}


In [38]:
# 不要ファイルを削除する

!rm -rf mecab-ipadic-neologd-0.0.7
%cd ..

/home/analysis01/src/kanjikana-model/dict/oss


## skk
単語辞書と異体字辞書，単漢字辞書を作成する。単漢字辞書は単語辞書から一文字の漢字を抜き出したもの。


In [39]:
%cd skk
!wget -nc -q "https://github.com/skk-dev/dict/raw/master/SKK-JISYO.jinmei" --no-check-certificate
!wget -nc -q "https://github.com/skk-dev/dict/raw/master/SKK-JISYO.fullname" --no-check-certificate
!wget -nc -q https://github.com/skk-dev/dict/raw/master/SKK-JISYO.itaiji --no-check-certificate

/home/analysis01/src/kanjikana-model/dict/oss/skk


In [40]:
# 姓名辞書を作成する
!python jinmei.py --infile SKK-JISYO.jinmei --outfile skk_jinmei.txt
!python fullname.py --infile SKK-JISYO.fullname --outfile skk_fullname.txt
!cat skk_jinmei.txt skk_fullname.txt > dict.txt


{
  "infile": "SKK-JISYO.jinmei",
  "outfile": "skk_jinmei.txt"
}
{
  "infile": "SKK-JISYO.fullname",
  "outfile": "skk_fullname.txt"
}


In [41]:
# 単漢字辞書を作成する
!python ../tankan.py --infile dict.txt --outfile tankanji.txt



{
  "infile": "dict.txt",
  "outfile": "tankanji.txt"
}


In [42]:
# 異体字辞書を作成する
!python itaiji_skk.py --infile SKK-JISYO.itaiji  --outfile itaiji.txt


{
  "infile": "SKK-JISYO.itaiji",
  "outfile": "itaiji.txt"
}


In [43]:
# 不要ファイルを削除する

%cd ..

/home/analysis01/src/kanjikana-model/dict/oss


## wikipedia
wikipediaからは，概要部分を抜き出して，文頭に　「漢字（読み）・・」　と記載されている場合に，漢字と読み仮名のペアを抜き出す。多少間違えている可能性があるので注意。


Wikipediaの構造解析から，Abstract部分に着目する

'''アルマ・マリア・マーラー＝ヴェルフェル'''（'''Alma Maria Mahler-Werfel''', [[1879年]][[8月31日]] – [[1964年]][[12月11日]]）は、[[オーストリア]]の作曲家[[グス
タフ・マーラー]]の妻。結婚前の姓はシンドラー（Schindler）。自身も作曲を行い、16>の歌曲が今日に残されている。華麗な男性遍歴で知られる。 


こんな形で入っている行をまずは抜き出す。     
- 見出しの最初に，'''人名'''
- ()の中に'''読み'''が入っている
- ()の中に[[yyyy年]]
- ()の中に[[mm月dd日]]


In [44]:
%cd wikipedia
!rm -rf jawiki-latest-pages-articles.xml.bz2
!wget -nc -q https://dumps.wikimedia.org/jawiki/latest/jawiki-latest-pages-articles.xml.bz2 --no-check-certificate



/home/analysis01/src/kanjikana-model/dict/oss/wikipedia


In [45]:
# Wikipediaから概要だけを抜き出す
!python extract_wiki.py --infile="jawiki-latest-pages-articles.xml.bz2" --outfile="abstract.txt"



{
  "infile": "jawiki-latest-pages-articles.xml.bz2",
  "outfile": "abstract.txt"
}
i=0
i=1000000
i=2000000
i=3000000
i=4000000
i=5000000
i=6000000
i=7000000
i=8000000
i=9000000
i=10000000
i=11000000
i=12000000
i=13000000
i=14000000
i=15000000
i=16000000
i=17000000
i=18000000
i=19000000
i=20000000
i=21000000
i=22000000
i=23000000
i=24000000
i=25000000
i=26000000
i=27000000
i=28000000
i=29000000
i=30000000
i=31000000
i=32000000
i=33000000
i=34000000
i=35000000
i=36000000
i=37000000
i=38000000
i=39000000
i=40000000
i=41000000
i=42000000
i=43000000
i=44000000
i=45000000
i=46000000
i=47000000
i=48000000
i=49000000
i=50000000
i=51000000
i=52000000
i=53000000
i=54000000
i=55000000
i=56000000
i=57000000
i=58000000
i=59000000
i=60000000
i=61000000
i=62000000
i=63000000
i=64000000
i=65000000
i=66000000
i=67000000
i=68000000
i=69000000
i=70000000
i=71000000
i=72000000
i=73000000
i=74000000
i=75000000
i=76000000
i=77000000
i=78000000
i=79000000
i=80000000
i=81000000
i=82000000
i=83000000
i=840000

In [46]:
# 漢字とカナのペアとなるものと推測されるものを抜き出す。
!python extract_name.py --outfile="abstract_name.txt" --infile="abstract.txt";


{
  "outfile": "abstract_name.txt",
  "infile": "abstract.txt"
}
i=0
i=10000
i=20000
i=30000
i=40000
i=50000
i=60000
i=70000
i=80000
i=90000
i=100000
i=110000
i=120000
i=130000
i=140000
i=150000
i=160000
i=170000
i=180000
i=190000
i=200000
i=210000
i=220000
i=230000
i=240000


In [47]:
# extract_name.pyによってWikipediaのダンプファイルから，概要部分が抜き出されたものに対して，漢字・アルファベットとカタカナのペアのうち，正しいと推測されるものを選択する。
!python select_name.py --infile abstract_name.txt --outfile wikiname.txt --ngfile ng.txt;


{
  "infile": "abstract_name.txt",
  "outfile": "wikiname.txt",
  "ngfile": "ng.txt"
}
i=0
i=10000
i=20000
i=30000
i=40000
i=50000
i=60000
i=70000
i=80000
i=90000
i=100000
i=110000
i=120000
i=130000
i=140000
i=150000
i=160000
i=170000
i=180000
i=190000
i=200000
i=210000


In [48]:
# select_name.pyによってWikipediaのダンプファイルから作成した漢字・アルファベットとカタカナのペアを単語単位に分割する
!python dict_name.py --infile wikiname.txt --outfile dict.txt

{
  "infile": "wikiname.txt",
  "outfile": "dict.txt"
}


In [49]:
# 
%cd ../..


/home/analysis01/src/kanjikana-model/dict


# ダウンロードしたデータから辞書を作成する

## オープンソース辞書データ
オープンソースの辞書から抜き出したデータで，oss.json,tankanji.json, itaiji.jsonを作成する


In [50]:
# oss.json
!rm -rf oss.json
!python merge2.py --infile oss/canna/dict.txt --jsonfile oss.json --dicname canna;
!python merge2.py --infile oss/freewnn/dict.txt --jsonfile oss.json --dicname freewnn;
!python merge2.py --infile oss/ipadic/dict.txt --jsonfile oss.json --dicname ipadic;
!python merge2.py --infile oss/kakasi/dict.txt --jsonfile oss.json --dicname kakasi;
!python merge2.py --infile oss/mozc/dict.txt --jsonfile oss.json --dicname mozc;
!python merge2.py --infile oss/neologd/dict.txt --jsonfile oss.json --dicname neologd;
!python merge2.py --infile oss/skk/dict.txt --jsonfile oss.json --dicname skk;

{
  "infile": "oss/canna/dict.txt",
  "jsonfile": "oss.json",
  "dicname": "canna"
}
{
  "infile": "oss/freewnn/dict.txt",
  "jsonfile": "oss.json",
  "dicname": "freewnn"
}
{
  "infile": "oss/ipadic/dict.txt",
  "jsonfile": "oss.json",
  "dicname": "ipadic"
}
{
  "infile": "oss/kakasi/dict.txt",
  "jsonfile": "oss.json",
  "dicname": "kakasi"
}
{
  "infile": "oss/mozc/dict.txt",
  "jsonfile": "oss.json",
  "dicname": "mozc"
}
{
  "infile": "oss/neologd/dict.txt",
  "jsonfile": "oss.json",
  "dicname": "neologd"
}
{
  "infile": "oss/skk/dict.txt",
  "jsonfile": "oss.json",
  "dicname": "skk"
}


In [51]:
# tankanji.json
!rm -rf tankanji.json

!python merge2.py --infile oss/canna/tankanji.txt --jsonfile tankanji.json --dicname canna;
!python merge2.py --infile oss/freewnn/tankanji.txt --jsonfile tankanji.json --dicname freewnn;
!python merge2.py --infile oss/ipadic/tankanji.txt --jsonfile tankanji.json --dicname ipadic;
!python merge2.py --infile oss/kakasi/tankanji.txt --jsonfile tankanji.json --dicname kakasi;
!python merge2.py --infile oss/mozc/tankanji.txt --jsonfile tankanji.json --dicname mozc;
!python merge2.py --infile oss/neologd/tankanji.txt --jsonfile tankanji.json --dicname neologd;
!python merge2.py --infile oss/skk/tankanji.txt --jsonfile tankanji.json --dicname skk;
!python merge2.py --infile oss/mj/tankanji.txt --jsonfile tankanji.json --dicname mj;

{
  "infile": "oss/canna/tankanji.txt",
  "jsonfile": "tankanji.json",
  "dicname": "canna"
}
{
  "infile": "oss/freewnn/tankanji.txt",
  "jsonfile": "tankanji.json",
  "dicname": "freewnn"
}
{
  "infile": "oss/ipadic/tankanji.txt",
  "jsonfile": "tankanji.json",
  "dicname": "ipadic"
}
{
  "infile": "oss/kakasi/tankanji.txt",
  "jsonfile": "tankanji.json",
  "dicname": "kakasi"
}
{
  "infile": "oss/mozc/tankanji.txt",
  "jsonfile": "tankanji.json",
  "dicname": "mozc"
}
{
  "infile": "oss/neologd/tankanji.txt",
  "jsonfile": "tankanji.json",
  "dicname": "neologd"
}
{
  "infile": "oss/skk/tankanji.txt",
  "jsonfile": "tankanji.json",
  "dicname": "skk"
}
{
  "infile": "oss/mj/tankanji.txt",
  "jsonfile": "tankanji.json",
  "dicname": "mj"
}


In [52]:
# itaiji.json
!rm -rf itaiji.json
!python merge2.py --infile oss/kakasi/itaiji.txt --jsonfile itaiji.json --dicname kakasi
!python merge2.py --infile oss/mozc/itaiji.txt --jsonfile itaiji.json --dicname mozc
!python merge2.py --infile oss/skk/itaiji.txt --jsonfile itaiji.json --dicname skk


{
  "infile": "oss/kakasi/itaiji.txt",
  "jsonfile": "itaiji.json",
  "dicname": "kakasi"
}
{
  "infile": "oss/mozc/itaiji.txt",
  "jsonfile": "itaiji.json",
  "dicname": "mozc"
}
{
  "infile": "oss/skk/itaiji.txt",
  "jsonfile": "itaiji.json",
  "dicname": "skk"
}


## Webデータ
wikipediaから抜き出したデータから，姓名の漢字とカナのペアで，seimei.jsonを作成する


In [53]:
# オープンソースの辞書のデータに入っていない漢字とカナのペアを抜き出す
!python oss/wikipedia/check_dict.py --infile oss/wikipedia/dict.txt --outfile oss/wikipedia/dict_add.txt --oss oss.json;


{
  "infile": "oss/wikipedia/dict.txt",
  "outfile": "oss/wikipedia/dict_add.txt",
  "oss": "oss.json"
}
total=115043,cnt=41393


In [54]:
!rm -rf seimei.json
!python merge2.py --infile oss/wikipedia/dict.txt --jsonfile seimei.json --dicname wikipedia


{
  "infile": "oss/wikipedia/dict.txt",
  "jsonfile": "seimei.json",
  "dicname": "wikipedia"
}


In [55]:
!date


2025年  5月 30日 金曜日 14:15:52 JST
